In [2]:
import json

with open (r"C:\Users\bhate\AppData\Local\Microsoft\Windows\INetCache\IE\ACJQHL0Q\parsed_resumes[1].json",'r',encoding='utf-8')as f:
    parsed_resumes = json.load(f)

In [1]:
from json_to_sqlite import ResumeDatabase

In [2]:
db = ResumeDatabase()

In [3]:
import sqlite3
import json

# Example: List of resume dictionaries
resume_data_list = parsed_resumes

def create_table(conn):
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS resumes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            email TEXT,
            phone TEXT,
            skills TEXT,
            education TEXT,
            experience TEXT,
            certifications TEXT,
            achievements TEXT,
            tech_stack TEXT,
            resume_id TEXT UNIQUE
        )
    ''')
    conn.commit()

def resume_exists(conn, resume_id):
    cursor = conn.cursor()
    cursor.execute('SELECT id FROM resumes WHERE resume_id = ?', (json.dumps(resume_id),))
    return cursor.fetchone() is not None

def insert_resume(conn, resume):
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO resumes (name, email, phone, skills, education, experience, certifications, achievements, tech_stack, resume_id)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        resume.get("name"),
        resume.get("email"),
        resume.get("phone"),
        json.dumps(resume.get("skills")),
        json.dumps(resume.get("education")),
        json.dumps(resume.get("experience")),
        json.dumps(resume.get("certifications")),
        json.dumps(resume.get("achievements")),
        json.dumps(resume.get("tech_stack")),
        json.dumps(resume.get("resume_id"))
    ))
    conn.commit()
    return cursor.lastrowid

# Main
if __name__ == "__main__":
    conn = sqlite3.connect("resumes.db")
    create_table(conn)

    for resume in resume_data_list:
        resume_id = resume.get("resume_id")
        if resume_id and not resume_exists(conn, resume_id):
            new_id = insert_resume(conn, resume)
            print(f"✅ Inserted new resume with ID: {new_id}")
        else:
            print(f"⚠️ Resume with ID {resume_id} already exists. Skipping...")

    conn.close()


✅ Inserted new resume with ID: 1
✅ Inserted new resume with ID: 2
✅ Inserted new resume with ID: 3
✅ Inserted new resume with ID: 4
